In [1]:
from pyspark.sql import SparkSession


# Crear una instancia de SparkSession:
spark = SparkSession\
        .builder\
        .master("local[*]")\
        .appName("Delay")\
        .getOrCreate()

In [2]:
ruta = r'C:\Users\MIS\Desktop\Estudio\4\RNA\archive(1)\Airlines.csv'

df = spark.read.csv(ruta, header=True, inferSchema=True)

df.show(10)

+---+-------+------+-----------+---------+---------+----+------+-----+
| id|Airline|Flight|AirportFrom|AirportTo|DayOfWeek|Time|Length|Delay|
+---+-------+------+-----------+---------+---------+----+------+-----+
|  1|     CO|   269|        SFO|      IAH|        3|  15|   205|    1|
|  2|     US|  1558|        PHX|      CLT|        3|  15|   222|    1|
|  3|     AA|  2400|        LAX|      DFW|        3|  20|   165|    1|
|  4|     AA|  2466|        SFO|      DFW|        3|  20|   195|    1|
|  5|     AS|   108|        ANC|      SEA|        3|  30|   202|    0|
|  6|     CO|  1094|        LAX|      IAH|        3|  30|   181|    1|
|  7|     DL|  1768|        LAX|      MSP|        3|  30|   220|    0|
|  8|     DL|  2722|        PHX|      DTW|        3|  30|   228|    0|
|  9|     DL|  2606|        SFO|      MSP|        3|  35|   216|    1|
| 10|     AA|  2538|        LAS|      ORD|        3|  40|   200|    1|
+---+-------+------+-----------+---------+---------+----+------+-----+
only s

In [3]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Flight: integer (nullable = true)
 |-- AirportFrom: string (nullable = true)
 |-- AirportTo: string (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Time: integer (nullable = true)
 |-- Length: integer (nullable = true)
 |-- Delay: integer (nullable = true)



In [4]:
df.groupBy('Airline').count().sort('count',ascending=True).show()

+-------+-----+
|Airline|count|
+-------+-----+
|     HA| 5578|
|     F9| 6456|
|     AS|11471|
|     OH|12630|
|     YV|13725|
|     B6|18112|
|     9E|20686|
|     FL|20827|
|     CO|21118|
|     UA|27619|
|     EV|27983|
|     XE|31126|
|     US|34500|
|     MQ|36605|
|     AA|45656|
|     OO|50254|
|     DL|60940|
|     WN|94097|
+-------+-----+



In [7]:
df.groupBy('Delay').count().sort('count',ascending=False).show()

+-----+------+
|Delay| count|
+-----+------+
|    0|299119|
|    1|240264|
+-----+------+



In [9]:
df.describe("Length","Time").show()

+-------+------------------+------------------+
|summary|            Length|              Time|
+-------+------------------+------------------+
|  count|            539383|            539383|
|   mean|132.20200673732765| 802.7289625368245|
| stddev| 70.11701559746582|278.04591081678984|
|    min|                 0|                10|
|    max|               655|              1439|
+-------+------------------+------------------+



In [10]:
df.crosstab('Delay', 'Airline').sort('Delay_airline').show()

+-------------+-----+-----+----+----+-----+-----+-----+----+-----+----+-----+----+-----+-----+-----+-----+-----+-----+
|Delay_Airline|   9E|   AA|  AS|  B6|   CO|   DL|   EV|  F9|   FL|  HA|   MQ|  OH|   OO|   UA|   US|   WN|   XE|   YV|
+-------------+-----+-----+----+----+-----+-----+-----+----+-----+----+-----+----+-----+-----+-----+-----+-----+-----+
|            0|12460|27920|7579|9653| 9161|33488|16728|3557|14552|3792|23863|9128|27494|18673|22909|28440|19331|10391|
|            1| 8226|17736|3892|8459|11957|27452|11255|2899| 6275|1786|12742|3502|22760| 8946|11591|65657|11795| 3334|
+-------------+-----+-----+----+----+-----+-----+-----+----+-----+----+-----+----+-----+-----+-----+-----+-----+-----+



In [12]:
df.crosstab('Delay', 'AirportFrom').sort('Delay_AirportFrom').show()

+-----------------+---+---+----+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+---+----+---+---+---+----+---+---+---+---+---+---+---+---+---+---+----+---+----+---+---+---+---+---+---+---+----+----+----+---+---+---+---+---+---+---+---+---+---+---+---+----+---+----+----+---+---+---+---+---+---+---+---+---+----+---+---+---+----+---+---+----+-----+-----+---+---+---+---+----+---+---+---+---+---+----+---+---+---+---+----+---+---+---+---+---+---+---+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+---+---+---+---+----+----+---+---+---+----+---+---+---+---+---+---+---+----+----+---+---+---+---+----+----+---+---+---+---+----+---+---+---+---+---+---+---+---+---+---+---+---+----+----+---+----+---+----+---+---+---+---+---+----+----+---+---+---+---+---+---+---+---+---+---+---+---+----+----+---+---+---+----+----+----+----+---+---+-----+---+---+---+---+----+----+---+----+----+---+---+---+----+---+---+---+---+---+---+---+---+---+---+

In [13]:
df.count()

539383

In [14]:
df.filter(df.Time<=15).count()

93

In [15]:
df.filter(df.Delay==0).count()

299119

In [16]:
df.groupBy('Flight').agg({'Time':'mean'}).show()

+------+------------------+
|Flight|         avg(Time)|
+------+------------------+
|  6466|476.28169014084506|
|   463|  933.578947368421|
|  1580| 744.0990099009902|
|  6357|            781.94|
|  6397| 819.0555555555555|
|  3997| 540.6962025316456|
|   496| 739.8803418803419|
|  1591| 889.5211267605633|
|  6658| 984.9672131147541|
|  1829| 802.5547445255474|
|   471| 823.9830508474577|
|  1088|            732.61|
|  2366| 618.1884057971015|
|  2142| 806.9157894736842|
|   833|        649.921875|
|  2122| 671.1724137931035|
|  1959| 757.8048780487804|
|  5300|             505.0|
|  4519| 530.8771929824561|
|  1238| 820.5603448275862|
+------+------------------+
only showing top 20 rows



In [25]:
df_new = df.drop('id')

In [26]:
df_new = df_new.withColumn('Length Scaled', df['Length'] / 655)
df_new.show()

+-------+------+-----------+---------+---------+----+------+-----+-------------------+
|Airline|Flight|AirportFrom|AirportTo|DayOfWeek|Time|Length|Delay|              Scale|
+-------+------+-----------+---------+---------+----+------+-----+-------------------+
|     CO|   269|        SFO|      IAH|        3|  15|   205|    1|0.31297709923664124|
|     US|  1558|        PHX|      CLT|        3|  15|   222|    1|0.33893129770992364|
|     AA|  2400|        LAX|      DFW|        3|  20|   165|    1|0.25190839694656486|
|     AA|  2466|        SFO|      DFW|        3|  20|   195|    1|0.29770992366412213|
|     AS|   108|        ANC|      SEA|        3|  30|   202|    0| 0.3083969465648855|
|     CO|  1094|        LAX|      IAH|        3|  30|   181|    1|0.27633587786259545|
|     DL|  1768|        LAX|      MSP|        3|  30|   220|    0|0.33587786259541985|
|     DL|  2722|        PHX|      DTW|        3|  30|   228|    0| 0.3480916030534351|
|     DL|  2606|        SFO|      MSP|     

In [34]:
df_new.groupBy('Airline').agg({'Time':'mean'}).sort("avg(Time)", ascending=False).show()

+-------+-----------------+
|Airline|        avg(Time)|
+-------+-----------------+
|     FL|833.1544629567388|
|     YV|832.2568306010929|
|     B6|825.5022636925795|
|     OO|811.2070083973415|
|     F9|807.0885997521685|
|     US| 805.075884057971|
|     WN|802.3435922505499|
|     AA|800.8028955668477|
|     EV|799.8396526462495|
|     DL|799.6361667213653|
|     UA|797.7448857670445|
|     AS|796.6563507976637|
|     HA|795.2260666905701|
|     OH|793.0209817893904|
|     9E|792.1733056173257|
|     XE|792.1371843474908|
|     MQ|791.4752356235487|
|     CO|790.3554787385169|
+-------+-----------------+



In [42]:
import matplotlib.pyplot as plt
import pandas as pd

sample_df = df_new.sample(withReplacement=False, fraction=0.5, seed=42)
pandas_df = sample_df.toPandas()


Py4JJavaError: An error occurred while calling o278.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 84.0 failed 1 times, most recent failure: Lost task 0.0 in stage 84.0 (TID 145) (WIN11-5490LT executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler$$Lambda$4321/137142533.apply(Unknown Source)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DA